In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('p2_reddit.csv')

In [3]:
data.head()

,Unnamed: 0,text,label
0,0,alway taught peopl like jordan peterson matt w...,0
1,1,don’t actual know qanon hundr leagu past stand...,0
2,2,f becom concern cousin’ f well relationship cl...,0
3,3,marri narcissist man littl decad final let go ...,0
4,4,f brotherinlaw alway total jackass marri siste...,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)
X_train = X_train.astype(str)
X_test = X_test.astype(str)

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, Attention
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
X_train_tagged_texts = []
for text in X_train:
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    X_train_tagged_texts.append(tags)
X_test_tagged_texts = []
for text in X_test:
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    X_test_tagged_texts.append(tags)
# Map POS tags to integers using NLTK's tagset
tagset = {"CC": 1, "CD": 2, "DT": 3, "EX": 4, "FW": 5, "IN": 6, "JJ": 7, "JJR": 8, "JJS": 9, "LS": 10, "MD": 11,
          "NN": 12, "NNS": 13, "NNP": 14, "NNPS": 15, "PDT": 16, "POS": 17, "PRP": 18, "PRP$": 19, "RB": 20,
          "RBR": 21, "RBS": 22, "RP": 23, "SYM": 24, "TO": 25, "UH": 26, "VB": 27, "VBD": 28, "VBG": 29,
          "VBN": 30, "VBP": 31, "VBZ": 32, "WDT": 33, "WP": 34, "WP$": 35, "WRB": 36}
X_train_tagged = []
for tagged_text in X_train_tagged_texts:
    tagged_text_int = []
    for word, tag in tagged_text:
        if tag in tagset:
            tagged_text_int.append(tagset[tag])
    X_train_tagged.append(tagged_text_int)

X_test_tagged = []
for tagged_text in X_test_tagged_texts:
    tagged_text_int = []
    for word, tag in tagged_text:
        if tag in tagset:
            tagged_text_int.append(tagset[tag])
    X_test_tagged.append(tagged_text_int)


max_length = max([len(seq) for seq in X_train_tagged])
max_length = max([len(seq) for seq in X_test_tagged])
# Pad sequences to a fixed length
# max_length = 12270
X_train_padded = pad_sequences(X_train_tagged, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_tagged, maxlen=max_length, padding='post')

In [8]:

# LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tagset)+1, output_dim=300, input_length=max_length))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=5))
model.add(Dense(64, activation='tanh'))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation='softmax'))  # 2 classes: generated by same method or not
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)
# train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32)


Epoch 1/15
26/26 [==============================] - 21s 527ms/step - loss: 0.3966 - accuracy: 0.7710
Epoch 2/15
26/26 [==============================] - 13s 516ms/step - loss: 0.1776 - accuracy: 0.9282
Epoch 3/15
26/26 [==============================] - 13s 505ms/step - loss: 0.1662 - accuracy: 0.9332
Epoch 4/15
26/26 [==============================] - 13s 505ms/step - loss: 0.1597 - accuracy: 0.9381
Epoch 5/15
26/26 [==============================] - 13s 509ms/step - loss: 0.1424 - accuracy: 0.9431
Epoch 6/15
26/26 [==============================] - 13s 509ms/step - loss: 0.1440 - accuracy: 0.9443
Epoch 7/15
26/26 [==============================] - 13s 512ms/step - loss: 0.1315 - accuracy: 0.9505
Epoch 8/15
26/26 [==============================] - 13s 516ms/step - loss: 0.1354 - accuracy: 0.9443
Epoch 9/15
26/26 [==============================] - 13s 514ms/step - loss: 0.1180 - accuracy: 0.9542
Epoch 10/15
26/26 [==============================] - 13s 516ms/step - loss: 0.1057 - accura

In [9]:
# tokenizer.fit_on_texts(X_test)
loss, acc = model.evaluate(X_test_padded, y_test)

7/7 [==============================] - 3s 113ms/step - loss: 0.1989 - accuracy: 0.9554


In [10]:
preds = model.predict(X_test_padded)

7/7 [==============================] - 2s 116ms/step


In [11]:
preds[0]

array([0.00153447, 0.9984655 ], dtype=float32)

In [12]:
preds_new = []
for i in preds:
  if(i[0]>i[1]):
    preds_new.append(0.0)
  else:
    preds_new.append(1.0)

In [13]:
preds_new[1]

1.0

In [14]:
y_test_array = np.array(y_test)
y_test_array[1]

1

In [15]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new)
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new)
recall = recall_score(y_test, preds_new)
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

F1 score =  0.9552238805970149
Precision =  0.96
Recall =  0.9504950495049505
Accuracy =  0.9554455445544554
